In [58]:
import pandas as pd
import numpy as np

In [59]:
data = pd.read_csv("krkopt.data", header=None)

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28056 entries, 0 to 28055
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       28056 non-null  object
 1   1       28056 non-null  int64 
 2   2       28056 non-null  object
 3   3       28056 non-null  int64 
 4   4       28056 non-null  object
 5   5       28056 non-null  int64 
 6   6       28056 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [61]:
data.columns = ["wkc", "wkr", "wrc", "wrr", "bkc", "bkr", "opt rank" ]

In [62]:
# data[["wkc", "wrc", "bkc", "opt rank"]] = data[["wkc", "wrc", "bkc", "opt rank"]].astype('string')

In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28056 entries, 0 to 28055
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   wkc       28056 non-null  object
 1   wkr       28056 non-null  int64 
 2   wrc       28056 non-null  object
 3   wrr       28056 non-null  int64 
 4   bkc       28056 non-null  object
 5   bkr       28056 non-null  int64 
 6   opt rank  28056 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [64]:
# X = data.iloc[:, 0:6]
# y = data['opt rank']

In [65]:
# X

In [66]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [67]:
train.shape, test.shape

((22444, 7), (5612, 7))

In [68]:
import tensorflow as tf

In [69]:
tf.__version__

'2.5.0'

In [70]:
import tensorflow_decision_forests as tfdf

In [71]:
np.version.version

'1.20.2'

In [72]:
train = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="opt rank")
test = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="opt rank")

In [73]:
train

<BatchDataset shapes: ({wkc: (None,), wkr: (None,), wrc: (None,), wrr: (None,), bkc: (None,), bkr: (None,)}, (None,)), types: ({wkc: tf.string, wkr: tf.int64, wrc: tf.string, wrr: tf.int64, bkc: tf.string, bkr: tf.int64}, tf.int64)>

In [74]:
model = tfdf.keras.RandomForestModel()

In [90]:
model.fit(train)

351/351 [==============================] - 0s 803us/step


In [91]:
model.summary()

Model: "random_forest_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (6):
	bkc
	bkr
	wkc
	wkr
	wrc
	wrr

No weights

Variable Importance: NUM_NODES:
    1. "wrr" 215559.000000 ################
    2. "wrc" 167230.000000 ############
    3. "bkr" 119192.000000 ########
    4. "bkc" 112909.000000 #######
    5. "wkc" 51481.000000 ##
    6. "wkr" 21590.000000 

Variable Importance: NUM_AS_ROOT:
    1. "bkr" 197.000000 ################
    2. "wkr" 85.000000 ######
    3. "wkc" 14.000000 
    4. "bkc"  4.000000 

Variable Importance: SUM_SCORE:
    1. "wrr" 2935321.383898 ################
    2. "wrc" 2826048.633256 ##############
    3. "bkr" 2720744.952189 #############
    4. "bkc" 2522831.245298

In [92]:
model.compile(metrics=["accuracy"])
print(model.evaluate(test))


88/88 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.7803
[0.0, 0.7802922129631042]


In [82]:
pred

0.0

In [94]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)

In [100]:
# List all the other available learning algorithms
# tfdf.keras.get_all_models()

# ? tfdf.keras.GradientBoostedTreesModel


# Create another model with specified hyper-parameters
model = tfdf.keras.GradientBoostedTreesModel(
    num_trees=500,
    growing_strategy="BEST_FIRST_GLOBAL",
    max_depth=8,
    split_axis="SPARSE_OBLIQUE"
    )
model.fit(train)



351/351 [==============================] - 0s 775us/step


In [101]:
# Evaluate the model
model.compile(metrics=["accuracy"])
print(model.evaluate(test))

88/88 [==============================] - 2s 24ms/step - loss: 0.0000e+00 - accuracy: 0.8886
[0.0, 0.8886315226554871]


In [99]:
tfdf.keras.GradientBoostedTreesModel?

Init signature: tfdf.keras.GradientBoostedTreesModel(*args, **kwargs)
Docstring:     
Gradient Boosted Trees learning algorithm.

A GBT (Gradient Boosted [Decision] Tree; https://statweb.stanford.edu/~jhf/ftp/trebst.pdf) is a set of shallow decision trees trained sequentially. Each tree is trained to predict and then "correct" for the errors of the previously trained trees (more precisely each tree predict the gradient of the loss relative to the model output).

Usage example:

```python
import tensorflow_decision_forests as tfdf
import pandas as pd

dataset = pd.read_csv("project/dataset.csv")
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(dataset, label="my_label")

model = tfdf.keras.GradientBoostedTreesModel()
model.fit(tf_dataset)

print(model.summary())
```

Attributes:
  task: Task to solve (e.g. Task.CLASSIFICATION, Task.REGRESSION,
    Task.RANKING).
  features: Specify the list and semantic of the input features of the model.
    If not specified, all the available featur